In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar', 'pH']]
target = wine['class']
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

와인 데이터셋 판다스로 블러오고 훈련세트와 테스트 세트로 나눔

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


cross_validate()를 사용하면 RandomForestClassifier의 성능을
교차 검증으로 평가하면서, n_jobs=-1로 병렬 처리하고
return_train_score=True로 훈련 점수까지 함께 확인하여
과대적합 여부를 쉽게 파악

In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


rf.fit(train_input, train_target)

rf : RandomForestClassifier 객체 (랜덤 포레스트 분류기)

fit() 메서드 : 주어진 학습 데이터를 기반으로 모델을 학습(학습된 모델 생성)

형식: estimator.fit(X, y)

X : 입력 특성 행렬, shape = (샘플 수, 특성 수)

y : 타깃 레이블 벡터, shape = (샘플 수,)

기능:

내부의 여러 결정 트리(Decision Tree)를 학습

각 트리가 학습 데이터 패턴을 학습

모델 객체(rf) 내부에 학습된 상태 저장

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


랜덤포레스트를 학습시키고, 각 특성의 중요도와 모델의 일반화 성능(OOB 점수)을 확인

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


ExtraTreesClassifier 모델을 생성하고, cross_validate를 사용해 학습 데이터로 교차검증을 수행

In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


ExtraTreesClassifier 모델을 학습하고, 각 특성의 중요도를 확인하는 코드

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


Gradient Boosting 분류기를 학습하고 교차검증으로 성능을 평가

In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                               random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


Gradient Boosting 분류기를 하이퍼파라미터를 조정해 학습하고 교차검증으로 평가하는 과정

500개의 트리를 만들고 학습률을 0.2로 설정한 Gradient Boosting 모델 생성

훈련과 검증 점수 평균 출력 → 모델 성능과 과대적합 여부 확인

In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


학습 데이터(train_input, train_target)로 Gradient Boosting 모델 학습

학습 후 각 특성이 예측에 기여한 상대적 중요도를 배열로 반환
 값이 클수록 모델이 해당 특성을 더 중요하게 사용

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores= cross_validate (hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


랜덤성을 고정한 히스토그램 기반 Gradient Boosting 모델 생성

학습 데이터를 사용해 교차검증 수행
return_train_score=True로 훈련 점수도 함께 계산

훈련 세트와 검증 세트의 평균 점수 출력 → 모델 성능과 과대적합 여부 확인

In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


학습 데이터(train_input, train_target)로 모델 학습

Permutation Importance: 한 번에 하나의 특성을 무작위로 섞고 모델 성능 변화를 측정

n_repeats=10 → 각 특성을 10번 섞어 평균 중요도 계산

n_jobs=-1 → 모든 CPU 코어 사용

각 특성의 순열 중요도의 평균값을 반환

값이 클수록 모델 예측에 더 중요한 특성

In [ ]:
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)


[0.05969231 0.20238462 0.049     ]


Permutation Importance: 각 특성을 무작위로 섞어 모델 성능 변화를 측정

test_input, test_target → 테스트 데이터를 사용하여 모델 일반화 성능 기반 평가

n_repeats=10 → 각 특성을 10번 섞어 평균 중요도 계산

n_jobs=-1 → 모든 CPU 코어 사용

각 특성의 순열 중요도 평균값 반환

In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9567059184812372 0.8783915747390243


tree_method='hist' → 히스토그램 기반 트리 학습, 큰 데이터셋에서 빠른 학습

random_state=42 → 결과 재현 가능

학습 데이터를 사용해 교차검증 수행

return_train_score=True → 훈련 점수도 계산

n_jobs=-1 → 모든 CPU 코어 사용

훈련 세트와 검증 세트의 평균 정확도 출력

훈련 점수와 검증 점수를 비교해 과대적합 여부 확인

In [ ]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


LightGBM 분류기 생성

random_state=42 → 결과 재현 가능

학습 데이터를 사용해 교차검증 수행

return_train_score=True → 훈련 점수도 계산

n_jobs=-1 → 모든 CPU 코어 사용

훈련 세트와 검증 세트의 평균 정확도 출력

훈련 점수와 검증 점수를 비교해 과대적합 여부 확인